# Torch tensor input and output mapping (V1 protocol)

This notebooks illustrates the how inputs and outputs are marshalled from the plug/webscript invocation to a tensor model.



It illustrates the behaviour of the [ml_adapter.torch.V1TorchMarshaller](https://github.com/waylayio/ml-adapters/blob/main/pkg/mla-torch/src/ml_adapter/torch/marshall.py) which is the default marshalling for the torch adapters `ml_adapter.torch.V1TorchAdapter` and `ml_adapter.torch.V1TorchNoLoadAdapter`. 

The encoding in json follows the [KServe V1 protocol](https://kserve.github.io/website/master/modelserving/data_plane/v1_protocol/).



In [1]:
import torch
import numpy as np
from ml_adapter.torch import V1TorchAdapter
from ml_adapter.torch import V1TorchNoLoadAdapter

from IPython.display import Markdown


### single tensor in/out

In [2]:
class Simple():
    """Simplest of torch models that doubles the input tensor"""
    def __call__(self, x: torch.Tensor):
        return x * 2

In [3]:
# call directly the model
Simple()(torch.tensor([1,2,3]))

tensor([2, 4, 6])

In [4]:
# a model adapter 
adapter = V1TorchNoLoadAdapter(model=Simple())

In [5]:
# supported marshalling from json/REST calls:
cases = [
    ('v1 instances', {'instances':[[1,2,3], [4,1,2]]} , 'single tensor as `instances`'),
    ('v1 instances dict named', {'instances':[{'x':[1,2,3]}, {'x':[4,1,2]}]} , 'a list of instances with named inputs'),
    ('v1 instances dict main', {'instances':[{'main':[1,2,3]}, {'main':[4,1,2]}]} , 'a list of instances with default inputs'),

    ('v1 inputs named ', {'inputs': { 'x': [[1,2,3], [4,1,2]]}} , 'a single named input tensor'),
    ('v1 inputs main ', {'inputs':{ 'main': [[1,2,3], [4,1,2]] } } , 'a single `main` input tensor'),
    ('v1 inputs main ignored y ', {'inputs':{ 'main': [[1,2,3], [4,1,2]] , 'y': [0,0,0]} } , 'non-mapped y input'),
    ('v1 inputs named ignored main ', {'inputs':{ 'main': [[1,2,3], [4,1,2]] , 'x': [0,0,0]} } , 'non-mapped main input'),
    ('v1 inputs named ignored y', {'inputs':{ 'x': [[1,2,3], [4,1,2]] , 'y': [0,0,0]} } , 'non-mapped additiona input input'),
    ('v1 inputs tensor', {'inputs':[[1,2,3], [4,1,2]]} , 'a single unnamed input tensor'),
]
display(Markdown('#### supported cases'))
display(Markdown(
    "| title | input | output | description |\n" +
    "| ----- | ----- | -----  | ----- |\n" +
    "".join([ f' | {case[0]} | {case[1]} | {await adapter.call(case[1])} | {case[2]} |\n'  for case in cases])
))

#### supported cases

| title | input | output | description |
| ----- | ----- | -----  | ----- |
 | v1 instances | {'instances': [[1, 2, 3], [4, 1, 2]]} | {'predictions': [[2, 4, 6], [8, 2, 4]]} | single tensor as `instances` |
 | v1 instances dict named | {'instances': [{'x': [1, 2, 3]}, {'x': [4, 1, 2]}]} | {'predictions': [[2, 4, 6], [8, 2, 4]]} | a list of instances with named inputs |
 | v1 instances dict main | {'instances': [{'main': [1, 2, 3]}, {'main': [4, 1, 2]}]} | {'predictions': [[2, 4, 6], [8, 2, 4]]} | a list of instances with default inputs |
 | v1 inputs named  | {'inputs': {'x': [[1, 2, 3], [4, 1, 2]]}} | {'outputs': [[2, 4, 6], [8, 2, 4]]} | a single named input tensor |
 | v1 inputs main  | {'inputs': {'main': [[1, 2, 3], [4, 1, 2]]}} | {'outputs': [[2, 4, 6], [8, 2, 4]]} | a single `main` input tensor |
 | v1 inputs main ignored y  | {'inputs': {'main': [[1, 2, 3], [4, 1, 2]], 'y': [0, 0, 0]}} | {'outputs': [[2, 4, 6], [8, 2, 4]]} | non-mapped y input |
 | v1 inputs named ignored main  | {'inputs': {'main': [[1, 2, 3], [4, 1, 2]], 'x': [0, 0, 0]}} | {'outputs': [0, 0, 0]} | non-mapped main input |
 | v1 inputs named ignored y | {'inputs': {'x': [[1, 2, 3], [4, 1, 2]], 'y': [0, 0, 0]}} | {'outputs': [[2, 4, 6], [8, 2, 4]]} | non-mapped additiona input input |
 | v1 inputs tensor | {'inputs': [[1, 2, 3], [4, 1, 2]]} | {'outputs': [[2, 4, 6], [8, 2, 4]]} | a single unnamed input tensor |


#### failures

In [6]:
# failure : wrong named inputs
try:
    await adapter.call({'inputs': { 'y':[2,3,4]}})
except Exception as exc:
    display(exc)

ValueError('Model invocation has unbound "x" input without default.')

In [7]:
# failure : input not numeric
try:
    await adapter.call({'inputs': { 'x': {'y':[2,3,4]} }})
except Exception as exc:
    display(exc)

RuntimeError('Could not infer dtype of dict')

### Multiple tensor input arguments

In [8]:
class ThreeArgs():
    """Three args, one optional."""
    def __call__(self, x: torch.Tensor, y:torch.Tensor, z:torch.Tensor=None):
        z = z if z is not None else torch.tensor([2]) 
        return (x + y) * z

In [9]:
ThreeArgs()(torch.tensor([1,2,3]),torch.tensor([4,5,6]),torch.tensor([0.1]))

tensor([0.5000, 0.7000, 0.9000])

In [10]:
ThreeArgs()(torch.tensor([1,2,3]),torch.tensor([4,5,6]))

tensor([10, 14, 18])

In [11]:
adapter = V1TorchNoLoadAdapter(model=ThreeArgs())

In [12]:
# supported marshalling from json/REST calls:
cases = [
    ('v1 instances dict named', {'instances':[
        {'x':[1,2,3], 'y':[4,5,6]}, 
        {'x':[4,1,2], 'y':[0,0,0]}
    ]} , 'a list of instances with named inputs'),
    ('v1 instances dict default', {'instances':[
        {'main':[1,2,3], 'y':[4,5,6]}, 
        {'main':[4,1,2], 'y':[0,0,0]}
    ]} , 'a list of instances with named inputs and default input'),
    ('v1 instances dict non-default', {'instances':[
        {'main':[1,2,3], 'y':[4,5,6], 'z': [0.1]}, 
        {'main':[4,1,2], 'y':[0,0,0], 'z':[-0.1]}
    ]} , 'a list of instances with named inputs '),
     ('v1 inputs named default', {'inputs': { 
         'x': [[1,2,3], [4,1,2]], 
         'y': [[1,2,3], [4,1,2]]
     }} , 'named input tensors'),
     ('v1 inputs named override', {'inputs': { 
         'x': [[1,2,3], [4,1,2]], 
         'y': [[1,2,3], [4,1,2]],
         'z': -1
     }} , 'named input tensors'),
]
display(Markdown('#### supported cases'))
display(Markdown(
    "| title | input | output | description |\n" +
    "| ----- | ----- | -----  | ----- |\n" +
    "".join([ f' | {case[0]} | {case[1]} | {await adapter.call(case[1])} | {case[2]} |\n'  for case in cases])
))
    

#### supported cases

| title | input | output | description |
| ----- | ----- | -----  | ----- |
 | v1 instances dict named | {'instances': [{'x': [1, 2, 3], 'y': [4, 5, 6]}, {'x': [4, 1, 2], 'y': [0, 0, 0]}]} | {'predictions': [[10, 14, 18], [8, 2, 4]]} | a list of instances with named inputs |
 | v1 instances dict default | {'instances': [{'main': [1, 2, 3], 'y': [4, 5, 6]}, {'main': [4, 1, 2], 'y': [0, 0, 0]}]} | {'predictions': [[10, 14, 18], [8, 2, 4]]} | a list of instances with named inputs and default input |
 | v1 instances dict non-default | {'instances': [{'main': [1, 2, 3], 'y': [4, 5, 6], 'z': [0.1]}, {'main': [4, 1, 2], 'y': [0, 0, 0], 'z': [-0.1]}]} | {'predictions': [[0.5, 0.699999988079071, 0.9000000357627869], [-0.4000000059604645, -0.10000000149011612, -0.20000000298023224]]} | a list of instances with named inputs  |
 | v1 inputs named default | {'inputs': {'x': [[1, 2, 3], [4, 1, 2]], 'y': [[1, 2, 3], [4, 1, 2]]}} | {'outputs': [[4, 8, 12], [16, 4, 8]]} | named input tensors |
 | v1 inputs named override | {'inputs': {'x': [[1, 2, 3], [4, 1, 2]], 'y': [[1, 2, 3], [4, 1, 2]], 'z': -1}} | {'outputs': [[-2, -4, -6], [-8, -2, -4]]} | named input tensors |


#### failures

In [13]:
# failure: with `instances` you can only provide dicts as list elements (name binding for each instance)
try:
    await adapter.call({'instances':[[1,2,3], [4,1,2]]})
except TypeError as exc:
    display(exc)

TypeError("ThreeArgs.__call__() missing 1 required positional argument: 'y'")

In [14]:
# each argument must will get a tensor with first dimension the instance count:
# e.g. here  `z` gets mapped to a `torch.tensor([-1,0.1])`, which fails
try:
    await adapter.call({'instances':[ 
        {'main':[1,2,3], 'y':[4,5,6], 'z':-1}, 
        {'main':[4,1,2], 'y':[0,0,0], 'z':+0.1}
    ]})
except RuntimeError as exc:
    display(exc)

RuntimeError('The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1')

In [15]:
### the same invalid invocation with `inputs` would be
try:
    await adapter.call({'inputs':{
        'main':[[1,2,3],[4,1,2]], 
         'y':[[4,5,6],[0,0,0]], 
         'z': [-1, 0.1]
    }})
except RuntimeError as exc:
    display(exc)

RuntimeError('The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1')

In [16]:
# in this examples,  `z` gets mapped to a `torch.tensor([[-1],[0.1]])`, which is accepted
await adapter.call({'instances':[ 
        {'main':[1,2,3], 'y':[4,5,6], 'z':[-1]}, 
        {'main':[4,1,2], 'y':[0,0,0], 'z':[+0.1]}
]})

{'predictions': [[-5.0, -7.0, -9.0],
  [0.4000000059604645, 0.10000000149011612, 0.20000000298023224]]}

In [17]:
# ... which is the same model input as
await adapter.call({'inputs':{
    'main':[[1,2,3],[4,1,2]], 
     'y':[[4,5,6],[0,0,0]], 
     'z': [[-1], [0.1]]
}})

{'outputs': [[-5.0, -7.0, -9.0],
  [0.4000000059604645, 0.10000000149011612, 0.20000000298023224]]}

### Multiple named output arguments

A model can output a dict of tensors. This will get mapped as a named outputs in the response:
* when using `instances`: as a list of dicts in `predictions`
* when using `inputs` as a dict of named lists in `outputs`

In [18]:
class DictOut():
    """Output is a dict of tensors."""
    def __call__(self, x: torch.Tensor):
         return { 'x':x, 'x2': x*x }


In [19]:
DictOut()(torch.tensor([0.2,-3]))

{'x': tensor([ 0.2000, -3.0000]), 'x2': tensor([0.0400, 9.0000])}

In [20]:
adapter_dict = V1TorchNoLoadAdapter(model=DictOut())

In [21]:
await adapter_dict.call({'instances': [2,-1]})

{'predictions': [{'x': 2, 'x2': 4}, {'x': -1, 'x2': 1}]}

In [22]:
await adapter_dict.call({'inputs': [2,-1]})

{'outputs': {'x': [2, -1], 'x2': [4, 1]}}

In [23]:
await adapter_dict.call({'inputs': {'x': 2}})

{'outputs': {'x': 2, 'x2': 4}}

### Additional output parameters

With the constructor argument `output_params=True` on the adapter, an additional second `parameters` output is supported, that is not treated by the tensor marshalling, and is rendered directly as `parameters` in the response.

In [24]:

class TupleOut():
    """Three args, one optional."""
    def __call__(self, x: torch.Tensor):
         return  { 'x':x, 'squares': x*x}, {'labels' : [ 'positive' if i>=0 else 'negative' for i in x.tolist() ]} 

In [25]:
adapter_tuple = V1TorchNoLoadAdapter(model=TupleOut(), output_params=True)

In [26]:
await adapter_tuple.call({'inputs': [2,-1]})

{'parameters': {'labels': ['positive', 'negative']},
 'outputs': {'x': [2, -1], 'squares': [4, 1]}}

In [27]:
await adapter_tuple.call({'instances': [2,-1]})

{'parameters': {'labels': ['positive', 'negative']},
 'predictions': [{'x': 2, 'squares': 4}, {'x': -1, 'squares': 1}]}

### Additional input parameters

The the `parameters` property of a V1 request gets directly mapped to the call method as optional named params, 
_without any marshalling of tensors_. 

If a parameter key is not mentioned in the model signature, it is ignored.


The following parameter keys are special, as they influence the encoding of the torch tensors during mashalling:
* `datatype`: a name for the scalar type, as defined by [KServe V1](https://kserve.github.io/website/master/modelserving/data_plane/v1_protocol/)
> `BOOL`, `UINT8`, `UINT16`, `UINT32`, `UINT64`,`INT8`,`INT16`,`INT32`,`INT64`,`FP16`,`FP32`,`FP64`, `BYTES`
* `dtype`: the scalar type used. This can be a string, numpy or tensor dtype: 
> `float64`, `float32`, `float16`, `complex64`, `complex128`,`int64`, `int32`, `int16`, `int8`, `uint8` and `bool`.


Note that
* the pytorch libraries do not support the `BYTES` datatype
* the _complex_ dtypes cannot be serialized to/from json without additional effort in the main webscript/plug script.

In [28]:
class CheckMapping():
    """Check dtype encoding and report all parameters used"""
    def __call__(self, x: torch.Tensor, **parameters):
         return  x, {'check':f'dtype:{x.dtype}; shape:{x.shape};layout:{x.layout}; device: {x.device}', **parameters}

In [29]:
check = CheckMapping()

In [30]:
check(torch.tensor([1,2,3]))

(tensor([1, 2, 3]),
 {'check': 'dtype:torch.int64; shape:torch.Size([3]);layout:torch.strided; device: cpu'})

In [31]:
check(torch.tensor([1,2,3]).to(dtype=float))

(tensor([1., 2., 3.], dtype=torch.float64),
 {'check': 'dtype:torch.float64; shape:torch.Size([3]);layout:torch.strided; device: cpu'})

In [32]:
adapter_check = V1TorchNoLoadAdapter(model=CheckMapping(), output_params=True)

In [33]:
await adapter_check({'inputs':[1,2,3,4]})

{'parameters': {'check': 'dtype:torch.int64; shape:torch.Size([4]);layout:torch.strided; device: cpu'},
 'outputs': [1, 2, 3, 4]}

In [34]:
await adapter_check({'inputs':[1,2,3,4], 'parameters': { 'dtype': 'int16' }})

{'parameters': {'check': 'dtype:torch.int16; shape:torch.Size([4]);layout:torch.strided; device: cpu',
  'dtype': 'int16'},
 'outputs': [1, 2, 3, 4]}

In [35]:
await adapter_check({'inputs':[1,2,3,4], 'parameters': { 'datatype':'FP64' }})

{'parameters': {'check': 'dtype:torch.float64; shape:torch.Size([4]);layout:torch.strided; device: cpu',
  'datatype': 'FP64'},
 'outputs': [1.0, 2.0, 3.0, 4.0]}

In [36]:
await adapter_check({'inputs':[1,2,3,4], 'parameters': { 'custom' :'special' }})

{'parameters': {'check': 'dtype:torch.int64; shape:torch.Size([4]);layout:torch.strided; device: cpu',
  'custom': 'special'},
 'outputs': [1, 2, 3, 4]}

In [37]:
# the marshaller supports complex types, but these cannot be simply mapped from/to json, so
# You will need to adapt the JSON (de)serialization class in when handling the reques/response ...
await adapter_check({'inputs':[1,2+3j,3,4], 'parameters': { 'hint':'please use special json serialization!' }})

{'parameters': {'check': 'dtype:torch.complex64; shape:torch.Size([4]);layout:torch.strided; device: cpu',
  'hint': 'please use special json serialization!'},
 'outputs': [(1+0j), (2+3j), (3+0j), (4+0j)]}

In [38]:
import importlib.metadata
print('versions used:')
for lib in ['waylay-ml-adapter-sdk','waylay-ml-adapter-torch','torch','waylay-sdk-core','waylay-sdk-registry','waylay-sdk-rules']:
    print(f'- {lib}: {importlib.metadata.version(lib)}')

versions used:
- waylay-ml-adapter-sdk: 0.0.9
- waylay-ml-adapter-torch: 0.0.9
- torch: 2.5.1
- waylay-sdk-core: 0.3.2
- waylay-sdk-registry: 2.17.1.20241025
- waylay-sdk-rules: 6.12.0.20241025
